<a href="https://colab.research.google.com/github/ialara/or-student/blob/main/lectures/Lanchester_With_Reinforcements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def attrition(friendly, enemy, factor): 
    return max(0,friendly - factor*enemy)

def reinforcements(period, forces, reinf, force_ratio, blocks, thresholds):
    activated = [0, 0] #reinforcements called up
    for iside, side in enumerate(sides):
        reinforce_details = ' '
        neg_red = 2*iside-1
        if blocks[iside][0] > 0 and np.sum(reinf[iside][max(0,period-blocks[iside][2]):period]) == 0:
            if force_ratio*neg_red >= thresholds[iside][0]*neg_red:
                reinforce_details = 'Force Ratio: {0:5.3f} {1:4.1f} '.format(force_ratio, thresholds[iside][0])
            if forces[iside][period] <= thresholds[iside][1]:
                reinforce_details = reinforce_details + 'Engaged: {0:5.3f} {1:4.1f}'.format(forces[iside][period], thresholds[iside][1])
        if reinforce_details != ' ':
            #print(side+' reinforces at', period, reinforce_details)
            blocks[iside][0] -= 1
            activated[iside] = blocks[iside][1]
    return blocks, activated

In [ ]:
red = []
red = np.append(red, 1500)
red_reinf = []

blue = []
blue = np.append(blue, 839)
blue_reinf = []

sides = ['red', 'blue']
blocks = [[5, 300, 70], [5, 300, 70]]
reinforce_thresholds = [[2.5, 0.8*red[0]],[4.0, 0.8*blue[0]]]
withdraw_thresholds = [[1.5, 0.7*red[0]],[10.0, 0.7*blue[0]]]

period = 0
while red[period] > 0 and blue[period] > 0:
    FR = red[period]/blue[period]
    #Reinforcements
    blocks, activated = reinforcements(period, (red, blue), (red_reinf, blue_reinf), FR, blocks, reinforce_thresholds)
    red_reinf = np.append(red_reinf, activated[0])
    blue_reinf = np.append(blue_reinf, activated[1])
    #Add prior reinforcements, if any
    if period >=70:
        blue[period] += blue_reinf[period-70]
        red[period] += red_reinf[period-70]
    #Atrrition calculation
    blue = np.append(blue, attrition(blue[period], red[period], 1/2048))
    red = np.append(red, attrition(red[period], blue[period], 1/512))

    period += 1
    
periods = np.arange(0, period)
plt.plot(periods, blue[0:period], c='b')
plt.plot(periods, red[0:period], c='r')
plt.title("Dewar Simulation")
plt.xlabel("Periods")
plt.ylabel("Remaining Forces")
plt.ylim([0, 1.1*np.max((red, blue))])
plt.show()